In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
from typing import List
from io import StringIO
import datetime as dt
import datetime as dt
import pandas as pd
import psycopg2
import pytz
import re
import sys
import warnings
from bball_utils import TEAM_CODE_DICT, generate_unique_game_id, generate_unique_player_id
from bs4 import BeautifulSoup
from io import StringIO
from psycopg2.extras import execute_values
from psycopg2.extensions import AsIs
from typing import List
from urllib.request import urlopen
import psycopg2

In [20]:
# Connect to postgres db
try:
    conn = psycopg2.connect(
        dbname='sports_data',
        user='postgres',
        password='postgres',
        host='localhost',
        port='5432'
    )
except:
    print("Failure to connect to database.")

query = 'SELECT * FROM basketball.all_games'

data = pd.read_sql_query(query, conn)

# Close the cursor and connection
conn.close()

C:\Users\14086\AppData\Local\Temp\ipykernel_20852\2951955221.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)


In [22]:
data

,date,start_time,visitor,visitor_points,home,home_points,ot,attendance,unique_id,created_at
0,2020-12-22,19:00:00,GSW,99,BRK,125,None,0.0,be073d8f99f87fc70b0c40d6b6d210d0,2024-03-24 19:57:05.676734
1,2020-12-22,22:00:00,LAC,116,LAL,109,None,0.0,c39c31f404ba02171ffcc5d1c37c25a2,2024-03-24 19:57:05.676734
2,2020-12-23,19:00:00,CHA,114,CLE,121,None,300.0,828fe5de1600c4869830bd2a002a6a48,2024-03-24 19:57:05.676734
3,2020-12-23,19:00:00,NYK,107,IND,121,None,0.0,4113cb78418c6c9357c9ca2404fc6545,2024-03-24 19:57:05.676734
4,2020-12-23,19:00:00,MIA,107,ORL,113,None,3396.0,f06a34abe2510ae3c1cd5d560e4f2cee,2024-03-24 19:57:05.676734
...,...,...,...,...,...,...,...,...,...,...
2481,2022-06-05,20:00:00,BOS,88,GSW,107,None,18064.0,863d2098081729d8c49eec80766208f8,2024-03-24 19:58:08.391675
2482,2022-06-08,21:00:00,GSW,100,BOS,116,None,19156.0,5cbd3ce5103bcd8056ab40f8b4fb0424,2024-03-24 19:58:08.391675
2483,2022-06-10,21:00:00,GSW,107,BOS,97,None,19156.0,f2bbe8b9c39ded9464492619796e1b8a,2024-03-24 19:58:08.391675
2484,2022-06-13,21:00:00,BOS,94,GSW,104,None,18064.0,c949a22a98057682e7ce20b3348c8f4b,2024-03-24 19:58:08.391675


In [3]:
# https://www.basketball-reference.com/leagues/NBA_2023_totals.html
# https://www.basketball-reference.com/leagues/NBA_%(year)s_games.html
# Nexet steps: do the same basically for each game (with location, result, etc.), will need unique ID probs
# then for each game, each row will be a player (have a player id and a game id attached)

In [4]:
dat

,date,start_time,visitor,visitor_points,home,home_points,ot,attendance,unique_id,created_at
0,2020-12-22,19:00:00,GSW,99,BRK,125,None,0,be073d8f99f87fc70b0c40d6b6d210d0,2024-03-24 19:57:05.676734


In [5]:
temp = dat.iloc[0]
date = temp.date
site = temp.home

In [10]:
def clean_stat_sheet_table(stat_sheet_table: pd.DataFrame) -> pd.DataFrame:
    '''
    Clean the stat table thats concatenated. 

    Args:
        stat_sheet_table (pd.DataFrame): The raw stat table.

    Returns:
        pd.DataFrame: Cleaned table.
    '''
    # Remove header rows
    stat_sheet_table = stat_sheet_table[~stat_sheet_table['Starters'].isin(['Reserves', 'Team Totals'])]
    
    # Ignore stats of DNP players
    stat_sheet_table = stat_sheet_table[stat_sheet_table['MP'] != 'Did Not Play']

    # Create a timedelta object representing the duration
    # Function to convert time string to timedelta
    def parse_time(time_str):
        minutes, seconds = map(int, time_str.split(':'))
        return dt.timedelta(minutes=minutes, seconds=seconds)
    stat_sheet_table['MP'] = stat_sheet_table['MP'].apply(parse_time)

    # Rename for unique ID
    stat_sheet_table = stat_sheet_table.rename(columns={'Starters': 'Player'})
    stat_sheet_table = stat_sheet_table.fillna(0)
    return stat_sheet_table

def get_stat_sheet_table(date: dt.date, site: str) -> pd.DataFrame:
    '''
    Get the whole stat_sheet_table of a given game, queried by date and home

    Args:
        date (dt.date): Desired date of game.
        site (str): Home team of game, in three letter code.

    Returns:
        pd.DataFrame: Gets the entire stat sheet for a given game.
    '''
    url = f'https://www.basketball-reference.com/boxscores/%(date)s0%(site)s.html' % {'date': date.strftime('%Y%m%d'), 'site': site}
    try:
        html = urlopen(url)
    except Exception as e:
        print(e)
    soup = BeautifulSoup(html, features='html.parser')

    tables = soup.find_all('table', {'id': re.compile('box-.*-game-basic')})

    # Extract three-letter codes from table IDs
    team_codes = [re.search(r'box-(\w+)-game-basic', table['id']).group(1) for table in tables]

    # Convert the tables into a string and wrap it with StringIO
    html_string = "\n".join(str(table) for table in tables)
    html_io = StringIO(html_string)
    stat_sheet_tables = pd.read_html(html_io)

    # Add Team, Opponent, Visitor, and Home teams
    for i in range(2):
        stat_sheet_tables[i] = stat_sheet_tables[i].droplevel(level=0, axis=1)
        stat_sheet_tables[i]['Team'] = team_codes[i]
        stat_sheet_tables[i]['Opponent'] = team_codes[1-i]
        stat_sheet_tables[i]['Visitor'] = team_codes[0]
        stat_sheet_tables[i]['Home'] = team_codes[1]
        stat_sheet_tables[i]['Date'] = date

    # Put it together into one table
    stat_sheet_table = pd.concat(stat_sheet_tables)
    stat_sheet_table = clean_stat_sheet_table(stat_sheet_table)

    # Get unique ID columns
    stat_sheet_table['Player_ID'] = stat_sheet_table.apply(generate_unique_player_id, axis=1)
    stat_sheet_table['Game_ID'] = stat_sheet_table.apply(generate_unique_game_id, axis=1)
    return stat_sheet_table

In [ ]:
# insert or update on table "stat_sheet" violates foreign key constraint "stat_sheet_game_id_fkey"
# DETAIL:  Key (game_id)=(d80c0d5974fca9ad1033e7e45def61e6) is not present in table "all_games".

In [12]:
get_stat_sheet_table(date, site).iloc[0]

Player                         Andrew Wiggins
MP                            0 days 00:31:14
FG                                          4
FGA                                        16
FG%                                      .250
3P                                          2
3PA                                         6
3P%                                      .333
FT                                          3
FTA                                         4
FT%                                      .750
ORB                                         0
DRB                                         2
TRB                                         2
AST                                         1
STL                                         0
BLK                                         1
TOV                                         4
PF                                          4
PTS                                        13
+/-                                       -28
Team                              

In [230]:
temp

date                                    2021-10-19
start_time                                19:30:00
visitor                                        BKN
visitor_points                                 104
home                                           MIL
home_points                                    127
ot                                            None
attendance                                   17341
unique_id         f6cf8a724f286683188247905be4e80c
created_at              2024-03-24 09:12:44.770992
Name: 0, dtype: object

In [17]:
def insert_stat_sheet_table(date: dt.date, site: str) -> None:
    query = '''
        INSERT INTO basketball.stat_sheet(
            player, minutes_played, field_goals, field_goals_attempted, field_goal_percentage, three_pointers, three_pointers_attempted, three_pointer_percentage, 
            free_throws, free_throws_attempted, free_throw_percentage, offensive_rebounds, defensive_rebounds, total_rebounds, assists, steals, blocks, turnovers, 
            personal_fouls, points, plus_minus, team, opponent, visitor, home, date, player_id, game_id
            )
        VALUES %s
        ON CONFLICT DO NOTHING
    '''

        # Connect to postgres db
    try:
        conn = psycopg2.connect(
            dbname='sports_data',
            user='postgres',
            password='postgres',
            host='localhost',
            port='5432'
        )
    except:
        print('Failure to connect to database.')

    stat_sheet_table = get_stat_sheet_table(date, site)

    row_tuples = [tuple(row) for row in stat_sheet_table.values]
    with conn.cursor() as cursor:
        try:
            execute_values(cursor, query, row_tuples)
            print('Successfully executed %(date)s, %(site)s game stat_sheet data' % {'date': date.strftime('%Y-%m-%d'), 'site': site})
        except Exception as e:
            print(e)

    # Commit and close connection
    conn.commit()
    conn.close()
    

In [18]:
insert_stat_sheet_table(date, site)

Successfully executed 2020-12-22, BRK game stat_sheet data


In [19]:
temp

date                                    2020-12-22
start_time                                19:00:00
visitor                                        GSW
visitor_points                                  99
home                                           BRK
home_points                                    125
ot                                            None
attendance                                       0
unique_id         be073d8f99f87fc70b0c40d6b6d210d0
created_at              2024-03-24 19:57:05.676734
Name: 0, dtype: object

In [217]:
data = {
    'Date': dt.date(2021, 10, 19),
    'Visitor': 'BKN',
    'Home': 'MIL'
}
generate_unique_game_id(pd.Series(data))

'f6cf8a724f286683188247905be4e80c'

In [201]:
generate_unique_game_id(temp.rename(index={'date': 'Date', 'visitor': 'Visitor', 'home': 'Home'}))

'f6cf8a724f286683188247905be4e80c'

In [182]:
query = '''
    INSERT INTO basketball.stat_sheet(
        player, minutes_played, field_goals, field_goals_attempted, field_goal_percentage, three_pointers, three_pointers_attempted, three_pointer_percentage, 
        free_throws, free_throws_attempted, free_throw_percentage, offensive_rebounds, defensive_rebounds, total_rebounds, assists, steals, blocks, turnovers, 
        personal_fouls, points, plus_minus, team, opponent, visitor, home, date, player_id, game_id
        )
    VALUES %s
    ON CONFLICT DO NOTHING
'''

    # Connect to postgres db
try:
    conn = psycopg2.connect(
        dbname='sports_data',
        user='postgres',
        password='postgres',
        host='localhost',
        port='5432'
    )
except:
    print('Failure to connect to database.')

stat_sheet_table = get_stat_sheet_table(date, site)

row_tuples = [tuple(row) for row in stat_sheet_table.values]
with conn.cursor() as cursor:
    try:
        execute_values(cursor, query, row_tuples)
        print('Successfully executed %(date)s, %(site)s stat_sheet data' % {'date': date.strftime('%Y%m%d'), 'site': site})
    except Exception as e:
        print(e)

# Commit and close connection
conn.commit()
conn.close()

Index(['Player', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       '+/-', 'Team', 'Opponent', 'Visitor', 'Home', 'Date'],
      dtype='object')
INSERT has more expressions than target columns
LINE 7: ...1-10-19'::date,'7f8f2c8f6c88e09a86bc5a0c63ea1e3c','35405bfb7...
                                                             ^



In [184]:
len(row_tuples[0])

28

In [ ]:
def get_games_between(start: dt.date, end: dt.date) -> pd.DataFrame:
    '''
    Queries PostgreSQL and gets all the entire table of games.

    Returns:
        pd.DataFrame: All of the NBA games.
    '''
    # Connect to postgres db
    try:
        conn = psycopg2.connect(
            dbname='sports_data',
            user='postgres',
            password='postgres',
            host='localhost',
            port='5432'
        )
    except:
        print("Failure to connect to database.")

    query = 'SELECT * FROM basketball.all_games ag WHERE ag.date < %(end)s and ag.date > %(start)s'
    all_game_data = pd.read_sql_query(query, conn, params={'start': start, 'end': end})

    # Close the connection
    conn.close()
    return all_game_data